# Assignment 2

Subash Sunku

In [4]:
#Imports are prefered to be all at the top
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression 
from sklearn import linear_model 
from sklearn.decomposition import PCA
import statistics 
import math 
from collections import OrderedDict 
import scipy.stats as scs
  

# 1. [30] (Smoothing) Write functions that use the cut() and qcut() functions provided by DataFrame to smooth data in a given column using the following methods. Apply these function on column F.

In [17]:
pd.options.display.float_format = '{:.04}' .format
#Read CSV to Dataframe:
df = pd.read_csv('hwk01.csv', index_col = 0)
def eq_binnin(df, col_name, binning_method, replace_type, depth):
    bins = int(len(df)/depth)
    
    if binning_method == 'equal_depth':
        df['G'] = pd.qcut(df[col_name].rank(method='first'),bins)
    elif binning_method == 'equal_width': 
        df['G'] = pd.cut(df[col_name],bins) 
        
    sets = []
    for i, j in enumerate(df.groupby('G')[col_name].groups.values()):
        sets.append(df[col_name].iloc[j].index.values)
    if replace_type == 'mean':
        for i,j in enumerate(df.groupby('G')[col_name].mean()):
            df.loc[sets[i],'F_mean'] = j
        return df['F_mean'].round(4)
    elif replace_type == 'boundary':
        for i,j in enumerate(zip(df.groupby('G')[col_name].max(),df.groupby('G')[col_name].min())):
            maxi = j[0]
            mini = j[1]
            for k in range(depth):
                if (abs(df.loc[sets[i][k],col_name] - mini) >= abs(df.loc[sets[i][k],col_name] - maxi)):
                    df.loc[sets[i][k],'F_boundary'] = maxi
                else:
                    df.loc[sets[i][k],'F_boundary'] = mini
        return df['F_boundary'].round(4)
    else:
        print("Replace method provided is not available, using default median:")
        for i,j in enumerate(df.groupby('G')[col_name].median()):
            df.loc[sets[i],'F_median'] = j
        return df['F_median'].round(4)
        
df

,A,B,C,D,E,F
0,a1,b1,964,-0.5886,9.755,6
1,a1,b2,5342,-1.355,31.34,7
2,a1,b1,7265,-0.3449,0.6031,3
3,a2,b2,5787,-0.729,10.15,1
4,a1,b2,2889,1.196,21.86,2
...,...,...,...,...,...,...
995,a3,b1,462,-1.526,8.703,1
996,a2,b2,7285,-0.7289,8.265,7
997,a1,b2,8728,-0.612,8.913,5
998,a2,b2,2143,0.4221,17.91,7


## 1.a Equal-depth binning with bin means for depth k, for example k = 100.

In [10]:
def eq_depth_mean(df,depth):
    bins = int(len(df)/depth)
    df['G'] = pd.qcut(df['F'].rank(method='first'),bins)
    sets = []
    for i, j in enumerate(df.groupby('G')['F'].groups.values()):
        sets.append(df['F'].iloc[j].index.values)

    for i,j in enumerate(df.groupby('G')['F'].mean()):
        df.loc[sets[i],'F_mean'] = j
    return df['F_mean'].round(4)

print('Column F after applying Equal-depth binning with bin means for depth 100: ')
#print(eq_depth_mean(df,100))
#You can use seperate function to display or using the common function
print(eq_binnin(df, 'F', 'equal_depth', 'mean', 100))

Column F after applying Equal-depth binning with bin means for depth 100: 
0     5.42
1     6.32
2     2.25
3     1.06
4     1.06
      ... 
995   1.06
996   7.43
997   5.42
998   7.43
999   5.42
Name: F_mean, Length: 1000, dtype: float64


## 1.b  Equal-depth binning with bin boundaries for depth k, for example k = 100.

In [15]:
def eq_depth_boundaries(df,depth):
    sets = []
    for i, j in enumerate(df.groupby('G')['F'].groups.values()):
        sets.append(df['F'].iloc[j].index.values)

    for i,j in enumerate(zip(df.groupby('G')['F'].max(),df.groupby('G')['F'].min())):
        maxi = j[0]
        mini = j[1]
        for k in range(100):
            if (abs(df.loc[sets[i][k],'F'] - mini) >= abs(df.loc[sets[i][k],'F'] - maxi)):
                df.loc[sets[i][k],'F_bound'] = maxi
            else:
                df.loc[sets[i][k],'F_bound'] = mini
        #print(j[0],j[1])
    return df['F_bound'].round(4)
print('Column F after applying Equal-depth binning with bin boundaries for depth 100: ')
#print(eq_depth_boundaries(df,100))
print(eq_binnin(df, 'F', 'equal_depth', 'boundary', 100))

Column F after applying Equal-depth binning with bin boundaries for depth 100: 
0     6.0
1     7.0
2     3.0
3     1.0
4     2.0
       ..
995   1.0
996   7.0
997   5.0
998   7.0
999   5.0
Name: F_boundary, Length: 1000, dtype: float64


## 1.c Equal-width binning with bin median for 10 bins.

In [14]:
def eq_depth_median(df,bins):
    df['H'] = pd.cut(df['F'],bins)
    sets = []
    for i, j in enumerate(df.groupby('H')['F'].groups.values()):
        sets.append(df['F'].iloc[j].index.values)

    for i,j in enumerate(df.groupby('H')['F'].median()):
        df.loc[sets[i],'F_median'] = j
    return df['F_median'].round(4)
print('Column F after applying Equal-width binning with bin median for 10 bins: ')
#print(eq_depth_median(df,10))
print(eq_binnin(df, 'F', 'equal_width', 'median', 100))

Column F after applying Equal-width binning with bin median for 10 bins: 
Replace method provided is not available, using default median:
0     6.0
1     7.0
2     3.0
3     1.0
4     1.0
       ..
995   1.0
996   7.0
997   5.0
998   7.0
999   5.0
Name: F_median, Length: 1000, dtype: float64


# 2. [30] (Data Reduction) Write a function that takes a DataFrame, a set of column names (of numeric columns), and an integer p (less than the total number of columns in the table), and use PCA method in Scikit-Learn (specifically, sklearn.decomposition.PCA) to reduce the set of columns into p new columns. Apply this function to reduce the columns {C, D, E, F} into two columns p1, and p2.


In [18]:
pd.options.display.float_format = '{:.04}' .format
def data_reduction(df,columns,p):
    if p > len(columns):
        raise ValueError('p value should be less than columns')
    x = df.loc[:, columns].values
    pca = PCA(n_components=p)
    principalComponents = pca.fit_transform(x)
    return pd.DataFrame(data = principalComponents, columns = ['p1', 'p2'])
data_reduction(df,('C','D','E','F'),2)

,p1,p2
0,4.221e+03,-5.917
1,-157.3,15.89
2,-2.08e+03,-14.74
3,-602.3,-5.26
4,2.296e+03,6.283
...,...,...
995,4.723e+03,-6.977
996,-2.1e+03,-7.082
997,-3.543e+03,-6.354
998,3.042e+03,2.29


# 3. [40] (Correlation) For this question, you will need to use packages scipy.stats


## 3.a Compute the covariance and the correlation coefficient for each pair of the numeric columns.


In [19]:
def covariance(df):
    return df[['C','D','E','F']].cov().round(4)
print("Covariance is: ")
covariance(df)

Covariance is: 


,C,D,E,F
C,8.146e+06,50.39,-421.3,99.65
D,50.39,1.003,-0.6001,0.0524
E,-421.3,-0.6001,101.0,-0.2313
F,99.65,0.0524,-0.2313,9.747


In [20]:
def correlation(df):
    return df[['C','D','E','F']].corr().round(4)
print("Correlation is: ")
correlation(df)

Correlation is: 


,C,D,E,F
C,1.0,0.0176,-0.0147,0.0112
D,0.0176,1.0,-0.0596,0.0167
E,-0.0147,-0.0596,1.0,-0.0074
F,0.0112,0.0167,-0.0074,1.0


## 3.b Use the crosstab() function to construct the contingency table for columns A and B, similar to the following sample, where the distinct values in attribute A are {a1, a2} and in attribute B are {b1, b2, b3} (this is just a sample and may not be the same as  the data in your data file). Write a sequence of Python code to perform the Pearson’s chi-square (χ 2 ) test of independence with a confidence level of 0.001 to determine if the two attributes are correlated. You should use stats.chi2() to get the χ 2 distribution. Print sufficient information to report the result of the test.

In [35]:
#https://codingdisciple.com/chi-squared-python.html
crosstab = pd.crosstab(df['B'],df['A'])
crosstab_all = pd.crosstab(df['B'],df['A'],margins = True)
#print(crosstab_all.iloc[0][0:3].values)
#print(crosstab_all.iloc[1][0:3].values)
chi2, p, dof, expected = scs.chi2_contingency(crosstab)
#print(chi2, p, dof, expected)
print("\nDegree of Freedom is: " + str(dof))
print("\nP-value of the test: " + str(p))
print("\nThe expected frequencies, based on the marginal sums of the table:\n " + str(expected))
alpha = 0.001
chi2_alpha = scs.chi2.ppf(1-alpha,dof)
print("\nchi2 value is: " + str(chi2.round(4)) + " \n\nchi2 with alpha 0.001 is: " + str(chi2_alpha.round(4)))
print("")
crosstab_all


Degree of Freedom is: 2

P-value of the test: 0.001192572517069232

The expected frequencies, based on the marginal sums of the table:
 [[180.846 189.711 220.443]
 [125.154 131.289 152.557]]

chi2 value is: 13.4633 

chi2 with alpha 0.001 is: 13.8155



A,a1,a2,a3,All
B,,,,
b1,174,170,247,591
b2,132,151,126,409
All,306,321,373,1000
